# Prediction

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model = "mohammad-osoolian/parsbert-finetuned")
# classifier = pipeline("sentiment-analysis", model = "mohammadhabp/finetuned-parsbert-uncased-ArmanEmo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/651M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
classifier.predict('حس خیلی خوبی دارم')

[{'label': 'HAPPY', 'score': 0.9411950707435608}]

In [ ]:
classifier.predict('امروز حال من خوب نیست')

[{'label': 'SAD', 'score': 0.887709379196167}]

In [ ]:
classifier.predict('نگرانم برایش اتفاقی افتاده باشد')

[{'label': 'FEAR', 'score': 0.8856727480888367}]

In [ ]:
def predict(inpath, classifier, outpath=None):
  sentences = []
  with open(inpath, 'r', encoding="utf-8-sig") as f:
    for line in f.readlines():
      sentences.append(line.strip())
  predictions = classifier.predict(sentences)
  if outpath != None:
    with open(outpath, 'w') as f:
      f.write('Label, Score\n')
      for i in range(len(sentences)):
        f.write(f"{predictions[i]['label']}, {predictions[i]['score']}\n")

  return predictions


In [ ]:
predict('predict_test.csv', classifier, outpath='results.csv')

[{'label': 'HAPPY', 'score': 0.9465934634208679},
 {'label': 'OTHER', 'score': 0.9452375769615173},
 {'label': 'SAD', 'score': 0.8910128474235535}]

# Evaluation

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
import numpy as np

classes = ['HAPPY', 'SAD', 'ANGRY', 'FEAR', 'SURPRISE', 'HATE', 'OTHER']
class2id = {classes[i]: i for i in range(len(classes))}
id2class = {i: classes[i] for i in range(len(classes))}

def sigmoid(X):
    return 1 / (1 + np.exp(-X))

def heaviside(X):
    return np.heaviside(X - 0.5, 0)

def onehot(ids, size=len(classes)):
  result = np.zeros((len(ids), size))
  result[np.arange(len(ids)), ids] = 1
  return result

def compute_metrics(preds, labels):
  """
      preds: one_hot of predictions
      labels: one_hot of labels
  """

  predictions = preds
  labels = labels

  f1 = f1_score(labels, predictions, average=None, zero_division=0.0)
  f1 = {f'f1_C{i}': f1[i] for i in range(len(f1))}
  f1_macro = f1_score(labels, predictions, average='macro', zero_division=0.0)
  recall = recall_score(labels, predictions, average=None, zero_division=0.0)
  recall = {f'recall_C{i}': recall[i] for i in range(len(recall))}
  recall_macro = recall_score(labels, predictions, average='macro', zero_division=0.0)
  precision = precision_score(labels, predictions, average=None, zero_division=0.0)
  precision = {f'precision_C{i}': precision[i] for i in range(len(precision))}
  precision_macro = precision_score(labels, predictions, average='macro', zero_division=0.0)
  accuracy = accuracy_score(labels, predictions)
  results = {'accuracy': accuracy, 'precision_macro': precision_macro, 'recall_macro': recall_macro, 'f1_macro': f1_macro, **f1, **recall, **precision}
  return results

In [ ]:
def evaluate(inpath, delim, classifier):
  sentences = []
  labels = []
  with open(inpath, 'r', encoding="utf-8-sig") as f:
    for line in f.readlines():
      sentence, label = line.strip().split(delim)
      labels.append(class2id[label])
      sentences.append(sentence)

  predictions = classifier.predict(sentences)
  preds = [class2id[predictions[i]['label']] for i in range(len(predictions))]

  labels = onehot(np.array(labels))
  preds = onehot(np.array(preds))

  return compute_metrics(preds, labels)


In [ ]:
evaluate('/content/Project_Test - testset.tsv', '\t', classifier)

{'accuracy': 0.7272727272727273,
 'precision_macro': 0.7645021645021645,
 'recall_macro': 0.55,
 'f1_macro': 0.5748890860692103,
 'f1_C0': 0.9,
 'f1_C1': 0.6956521739130436,
 'f1_C2': 0.5714285714285715,
 'f1_C3': 0.6666666666666666,
 'f1_C4': 0.8571428571428571,
 'f1_C5': 0.0,
 'f1_C6': 0.33333333333333337,
 'recall_C0': 1.0,
 'recall_C1': 1.0,
 'recall_C2': 0.4,
 'recall_C3': 0.5,
 'recall_C4': 0.75,
 'recall_C5': 0.0,
 'recall_C6': 0.2,
 'precision_C0': 0.8181818181818182,
 'precision_C1': 0.5333333333333333,
 'precision_C2': 1.0,
 'precision_C3': 1.0,
 'precision_C4': 1.0,
 'precision_C5': 0.0,
 'precision_C6': 1.0}

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Project_Test - testset.tsv', sep='\t')
df.to_csv('test.csv')

In [ ]:
evaluate('/content/Project_Test - testset.tsv', '\t', classifier)

{'accuracy': 0.6363636363636364,
 'precision_macro': 0.6224489795918366,
 'recall_macro': 0.5071428571428571,
 'f1_macro': 0.5302721088435375,
 'f1_C0': 0.8571428571428571,
 'f1_C1': 0.5333333333333333,
 'f1_C2': 0.5714285714285715,
 'f1_C3': 0.6666666666666666,
 'f1_C4': 0.75,
 'f1_C5': 0.0,
 'f1_C6': 0.3333333333333333,
 'recall_C0': 1.0,
 'recall_C1': 0.5,
 'recall_C2': 0.4,
 'recall_C3': 0.5,
 'recall_C4': 0.75,
 'recall_C5': 0.0,
 'recall_C6': 0.4,
 'precision_C0': 0.75,
 'precision_C1': 0.5714285714285714,
 'precision_C2': 1.0,
 'precision_C3': 1.0,
 'precision_C4': 0.75,
 'precision_C5': 0.0,
 'precision_C6': 0.2857142857142857}